## Import modules
#### chromadb:
#### ollama:


In [1]:
import chromadb
import ollama

In [2]:
chroma_client = chromadb.Client()


##### Delete Collections

In [ ]:
#chroma_client.delete_collection(name="feedback_collection") # Delete a collection and all associated embeddings, documents, and metadata. ⚠️ This is destructive and not reversible
#chroma_client.delete_collection(name="python_collection") # Delete a collection and all associated embeddings, documents, and metadata. ⚠️ This is destructive and not reversible


##### Create Feedback Collection

In [3]:
feedback_collection = chroma_client.create_collection(name="feedback_collection")

In [4]:
import re
import os

base_path = "/Users/valeriaenriquezlimon/Documents/meta-h-generator/llm-metaheuristics/experiments-custumhys"
folder_name = "ollama_output_Rastrigin(5)_20241121_000015"
file_name = "execution_optuna_result_0.txt"
metaheuristic_file  = "/Users/valeriaenriquezlimon/Documents/meta-h-generator/llm-metaheuristics/experiments-custumhys/ollama_output_Rastrigin(5)_20241121_000015/execution_iteration_0.py"

# Construct the file path
file_path = os.path.join(base_path, folder_name, file_name)
with open(file_path, 'r', encoding='utf-8') as file:
    code_file = file.read()


with open(metaheuristic_file, 'r', encoding='utf-8') as file:
    metaheuristic_code_file = file.read()

hyperparameters_pattern = r"Mejores hiperparámetros encontrados:\n({.*?})"
performance_pattern = r"Mejor rendimiento encontrado:\n([\d.]+)"

hyperparameters_match = re.search(hyperparameters_pattern, code_file, re.DOTALL)
hyperparameters_dict = eval(hyperparameters_match.group(1)) if hyperparameters_match else None

performance_match = re.search(performance_pattern, code_file)
performance_found = float(performance_match.group(1)) if performance_match else None

if hyperparameters_dict:
    radius = float(hyperparameters_dict.get('radius', 0))
    angle = float(hyperparameters_dict.get('angle', 0))
    sigma = float(hyperparameters_dict.get('sigma', 0))
    factor = float(hyperparameters_dict.get('factor', 0))
    self_conf = float(hyperparameters_dict.get('self_conf', 0))
    swarm_conf = float(hyperparameters_dict.get('swarm_conf', 0))
    version = hyperparameters_dict.get('version', '')
    distribution = hyperparameters_dict.get('distribution', '')

    
    print(f"radius: {radius}")
    print(f"angle: {angle}")
    print(f"sigma: {sigma}")
    print(f"factor: {factor}")
    print(f"self_conf: {self_conf}")
    print(f"swarm_conf: {swarm_conf}")
    print(f"version: {version}")
    print(f"distribution: {distribution}")

print(f"Performance found: {performance_found}")


FileNotFoundError: [Errno 2] No such file or directory: '/Users/valeriaenriquezlimon/Documents/meta-h-generator/llm-metaheuristics/experiments-custumhys/ollama_output_Rastrigin(5)_20241121_000015/execution_optuna_result_0.txt'

In [6]:
number_iteration = "id_0"
parameters  = f"""radius": {radius},"angle": {angle},"sigma": {sigma},"factor": {factor},"self_conf": {self_conf},"swarm_conf": {swarm_conf},"version": {version},"distribution": {distribution}, "performance_found": {performance_found}"""
print(parameters)
feedback_collection.add(
    documents=[metaheuristic_code_file, parameters],
    metadatas=[{'source': 'metaheuristic_code_file'}, {'source': 'parameters'}],
    ids=["id_0", "id_1"]
)

radius": 0.5405117232957318,"angle": 14.408693167938573,"sigma": 0.046498549162240936,"factor": 0.11779253652571474,"self_conf": 2.951026835950703,"swarm_conf": 2.7883600653077947,"version": inertial,"distribution": uniform, "performance_found": 7.955976169162414


In [ ]:
feedback_collection.peek()

In [25]:
# all-minilm:latest
output = ollama.embeddings(
        prompt="give me all parameters",
        model="all-minilm:latest",
)
results = feedback_collection.query (
        query_embeddings=[output["embedding"]],
        n_results=1
)
data = results['documents'][0][0]
print(data)



radius": 0.5405117232957318,"angle": 14.408693167938573,"sigma": 0.046498549162240936,"factor": 0.11779253652571474,"self_conf": 2.951026835950703,"swarm_conf": 2.7883600653077947,"version": inertial,"distribution": uniform, "performance_found": 7.955976169162414


##### Create Metaheuristic Collection

In [5]:
python_collection = chroma_client.create_collection(name="python_collection")
# /Users/valeriaenriquezlimon/Documents/meta-h-generator/llm-metaheuristics/operators_builder

In [6]:
import os
import ollama  # Replace with the actual library you use for embeddings

# Define the directory containing Python files
python_files_directory = '/Users/valeriaenriquezlimon/Documents/meta-h-generator/llm-metaheuristics/metaheuristic_builder'

# Iterate through all items in the directory
for d in os.listdir(python_files_directory):
    file_path = os.path.join(python_files_directory, d)  # Full path to the file
    if os.path.isfile(file_path):  # Check if it's a file
        try:
            # Open and read the file
            with open(file_path, 'r') as file:
                file_content = file.read()

            # Generate embeddings for the file content
            response = ollama.embeddings(model="all-minilm:latest", prompt=file_content)
            embedding = response.get("embedding")

            # Add embedding to the collection if valid
            if embedding:
                python_collection.add(
                    ids=[d],
                    embeddings=[embedding],
                    documents=[file_content],
                    metadatas=[{"filename": d}]
                )
                print(f"Added {d} to the collection")
            else:
                print(f"Warning: Empty embedding generated for {d}")

        except Exception as e:
            print(f"Error processing {d}: {e}")
    else:
        # Skip if it's not a file
        print(f"Skipping {d}, not a file")


Added metaheuristic_template.txt to the collection
Added optuna_template.txt to the collection
Added operators.txt to the collection


In [ ]:
python_collection.peek()

##### Create Operators Collection

In [7]:
operators_collection = chroma_client.create_collection(name="operators_collection")
# 

In [8]:
import os
import ollama  # Replace with the actual library you use for embeddings

# Define the directory containing Python files
operators_files_directory = "/Users/valeriaenriquezlimon/Documents/meta-h-generator/llm-metaheuristics/operators_builder" 

# Iterate through all items in the directory
for d in os.listdir(operators_files_directory):
    file_path = os.path.join(operators_files_directory, d)  # Full path to the file
    if os.path.isfile(file_path):  # Check if it's a file
        try:
            # Open and read the file
            with open(file_path, 'r') as file:
                file_content = file.read()

            # Generate embeddings for the file content
            response = ollama.embeddings(model="all-minilm:latest", prompt=file_content)
            embedding = response.get("embedding")

            # Add embedding to the collection if valid
            if embedding:
                operators_collection.add(
                    ids=[d],
                    embeddings=[embedding],
                    documents=[file_content],
                    metadatas=[{"filename": d}]
                )
                print(f"Added {d} to the collection")
            else:
                print(f"Warning: Empty embedding generated for {d}")

        except Exception as e:
            print(f"Error processing {d}: {e}")
    else:
        # Skip if it's not a file
        print(f"Skipping {d}, not a file")


Added firefly_dynamic.txt to the collection
Added differential_mutation.txt to the collection
Added random_sample.txt to the collection
Added gravitational_search.txt to the collection
Added central_force_dynamic.txt to the collection
Added genetic_crossover.txt to the collection
Added random_search.txt to the collection
Added random_flight.txt to the collection
Added spiral_dynamic.txt to the collection
Added genetic_mutation.txt to the collection
Added swarm_dynamic.txt to the collection
Added local_random_walk to the collection


In [9]:
data_feedback = ["central_force_dynamic"]

In [20]:

# Probably not useful
# all-minilm:latest
output_metaheuristics = ollama.embeddings(
        prompt=f"Do not: {data_feedback}",
        model="all-minilm:latest",
)
results_metaheuristics = python_collection.query (
        query_embeddings=[output_metaheuristics["embedding"]],
        n_results=2
)
#data_metaheuristics = results_metaheuristics['documents'][0][0]
results_metaheuristics
#print(data_metaheuristics)


{'ids': [['operators.txt', 'metaheuristic_template.txt']],
 'embeddings': None,
 'documents': [['Remember to write the operators\' names, virables\' names and selectors\' names exactly as shown down below. For example, the operators\' names are ALL ALWAYS written in lower case and with a \'_\' instead of a space between words.\nSo write the parameters\' exactly as shown down below. \n\nThese are the parameters to take, (REMEMBER that each operaotr must have its own selector, and that depending on the selected operator, YOU MUST ONLY USE USE ONE VARIABLE PER PARAMETER, DO NOT USE THE WHOLE ARRAY, and write the variable without an array format, but as a float or string format):\nAlso consider, that if the dimension is bigger than 3, you must select selectors that have more space, such as \'all\'.\n{\n  "random_search": {  # operator\n    { # parameters\n      "scale": 1.0 or 0.01,\n      "distribution": "uniform" or "gaussian" or "levy"\n    },\n    selector: "greedy" or "all" or"metropo

##### Experiment # 1
#####  Trying to get ollama to answer

In [44]:
data_feedback = ["central_force_dynamic", 'differential_mutation',
   "random_search"]

In [50]:
from pathlib import Path

repo_root = Path.cwd()  # This assumes you're running the script from the repo root

# Define the rest of the path relative to the repository root
relative_path = Path("outputs-results/ollama_output_Rastrigin_15_20250108_165905/execution_iteration_0.py")

# Construct the full path
execution_path = repo_root / relative_path
try:
    with open(execution_path, 'r', encoding='utf-8') as file:
        content = file.read()  # Reads the entire file as a string
        print(content)  # Output the file content
except FileNotFoundError:
    print(f"File not found: {execution_path}")
except Exception as e:
    print(f"An error occurred: {e}")


File not found: /Users/valeriaenriquezlimon/Documents/meta-h-generator/llm-metaheuristics/experiments-custumhys/outputs-results/ollama_output_Rastrigin_15_20250108_165905/execution_iteration_0.py


In [52]:
# Correct base directory
base_path = Path("/Users/valeriaenriquezlimon/Documents/meta-h-generator/llm-metaheuristics/outputs-results")

# Construct the full path
relative_path = "ollama_output_Rastrigin_15_20250108_165905/execution_iteration_0.py"
execution_path = base_path / relative_path

print("Constructed Path:", execution_path)
print("Does the file exist?", execution_path.exists())
with execution_path.open('r', encoding='utf-8') as file:
    content = file.read()
    print(content)

Constructed Path: /Users/valeriaenriquezlimon/Documents/meta-h-generator/llm-metaheuristics/outputs-results/ollama_output_Rastrigin_15_20250108_165905/execution_iteration_0.py
Does the file exist? True
# Name: Hybrid Metaheuristic with Adaptive Strategies

# Code:
import sys
from pathlib import Path
import numpy as np
project_dir = Path(__file__).resolve().parents[2]  # Remember to write well this line: 'project_dir = Path(__file__).resolve().parents[2]'
sys.path.insert(0, str(project_dir))
import benchmark_func as bf
import metaheuristic as mh

fun = bf.Rastrigin(15)  # This is the selected problem, the problem may vary depending on the case.
prob = fun.get_formatted_problem()

heur = [
    (  # Search operator 1
        'swarm_dynamic',
        {
            'factor': 0.7,
            'self_conf': 2.54,
            'swarm_conf': 2.56,
            'version': 'inertial',
            'distribution': 'uniform'
        },
        'probabilistic'
    ),
    (
        'random_sample',
     

In [53]:
all_operators_array = ["random_search", "central_force_dynamic", "differential_mutation", "firefly_dynamic", "genetic_crossover", "genetic_mutation", "gravitational_search", "random_flight", "local_random_walk", "random_sample", "spiral_dynamic", "swarm_dynamic"]

In [54]:

# 
output = ollama.generate(
    model="qwen2.5-coder:latest",
    prompt=f"""
    Taking into account this metaheuristic created: {content} 
    Generate me an array such as called 'all_operators_array', such as: {all_operators_array}, 
    but without those metaheuristics encountered.
    DO NOT USE ```python and do NOT use any markdown code or use the triple backticks  (```) anywhere in your response, and do not respond anything else.
    """
)

print(output['response'])
answer = output['response']
print(answer)


all_operators_array = ['central_force_dynamic', 'differential_mutation', 'firefly_dynamic', 'genetic_crossover', 'genetic_mutation', 'gravitational_search', 'random_flight', 'local_random_walk', 'spiral_dynamic']
all_operators_array = ['central_force_dynamic', 'differential_mutation', 'firefly_dynamic', 'genetic_crossover', 'genetic_mutation', 'gravitational_search', 'random_flight', 'local_random_walk', 'spiral_dynamic']


In [37]:

# Probably not useful
# all-minilm:latest
output_metaheuristics = ollama.embeddings(
        prompt=f"Create a metaheuristic with even two, three or more operators: {answer}",
        model="all-minilm:latest",
)
results_metaheuristics = operators_collection.query (
        query_embeddings=[output_metaheuristics["embedding"]],
        n_results=3
)
#data_metaheuristics = results_metaheuristics['documents'][0][0]
results_metaheuristics
#print(data_metaheuristics)


{'ids': [['differential_mutation.txt',
   'central_force_dynamic.txt',
   'random_search.txt']],
 'embeddings': None,
 'documents': [['  "differential_mutation": { # operator\n    { # parameters\n      "expression": "rand" or "best" or "current" or  "current-to-best" or "rand-to-best" or "rand-to-best-and-current",\n      "num_rands": 1,\n      "factor": 1.0\n    },\n    selector: "greedy" or "all" or"metropolis" or"probabilistic"\n}',
   '"central_force_dynamic": {  # operator\n    { # parameters\n      "gravity": 0.001,\n      "alpha": 0.01,\n      "beta": 1.5,\n      "dt": 1.0\n    },\n    selector: "greedy" or "all" or"metropolis" or"probabilistic"\n}',
   '"random_search": {  # operator\n    { # parameters\n      "scale": 1.0 or 0.01,\n      "distribution": "uniform" or "gaussian" or "levy"\n    },\n    selector: "greedy" or "all" or"metropolis" or"probabilistic"\n}']],
 'uris': None,
 'data': None,
 'metadatas': [[{'filename': 'differential_mutation.txt'},
   {'filename': 'centra

In [ ]:
ollama.list

<bound method Client.list of <ollama._client.Client object at 0x14720ed80>>

In [ ]:
output = ollama.generate(
   model = "myqwen2.5:latest",
   prompt = f"""Using these parametes {data} and this metaheuristic data {data_metaheuristics}
   , create another metaheuristic with a better performance, you must use the given parameters in {data} because
   those are the ones that were genereted in an optimized function,
   do not use markdown code anywhere in the project, take into account the problem fun = bf.Rastrigin(5).
   """
)

print(output['response'])

# Name: EnhancedInertialSwarmOptimization

# Code:
import sys
from pathlib import Path

project_dir = Path(__file__).resolve().parents[2] # Remember to write well this line: 'project_dir = Path(__file__).resolve().parents[2]'
sys.path.insert(0, str(project_dir))
import benchmark_func as bf
import metaheuristic as mh

fun = bf.Rastrigin(5) # This is the selected problem, the problem may vary depending on the case.
prob = fun.get_formatted_problem()

heur = [
    (  # Search operator 1
        'swarm_dynamic',
        {
            'factor': 0.11779253652571474,
            'self_conf': 2.951026835950703,
            'swarm_conf': 2.7883600653077947,
            'version': "inertial",
            'distribution': "uniform"
        },
        'probabilistic'
    ),
    (
        'local_random_walk',
        {
            'probability': 0.75,
            'scale': 1.0,
            'distribution': "uniform"
        },
        'probabilistic'
    )
]

met = mh.Metaheuristic(prob, heur, num_ite